# Histograms

This section covers histogram handling. This is guided exercise which starts by reading a file with plain CMS data with the objective to make some histograms from the collisions stored inside. 

The file is the same one as the previous tutorial, which corresponds with a very small fraction of data from the ```/DoubleMuon/``` dataset of 2022 data i.e. collisions collected by muon triggers.

## Preview: Load input file

The file is loaded following the routines already explained in the previous tutorial. We access the TTree object and we initialize a new variable ```ttree```with it. We also count the total number of events into a variable ```nEvents```.

In [ ]:
import ROOT
import math

# Read the file and get the TTree
file = ROOT.TFile.Open("/eos/user/f/fernance/standard-TutoriasFiles/43718dea-5cd4-48a7-b73b-df168edf1fac.root")
ttree = file.Get('Events')
nEvents = ttree.GetEntries()

## 1 - Histogram initialization

We first create the histogram objects. Each histogram object is defined by a name (cpp object label), the title which can be optionally given to the histograms, the number of bins and the minimium and maximum values that the histogram shows.

In the future, each time that we put data in this histogram, it will be counted in the corresponding bin. We will be able to access the number of events that filled each one of the bins. However, it is important to take in mind that the bounds of the histograms **can't** be changed once created, once the histogram is filled.

Here we define both 1-dimensional (one variable) and 2-dimensional histograms:

In [ ]:
# Define 1-dimensional simple histograms: TH1F(key, title, nbins, xmin, xmax)

hnMuon       = ROOT.TH1F("hnMuon", "Number of muons", 10, 0, 10)       # Number of muons
hmuon_pt     = ROOT.TH1F("hmuon_pt", "Muon pT", 100, 0, 100)           # Muon pt
hmuon_eta    = ROOT.TH1F("hmuon_eta", "Muon eta", 100, -2.6, 2.6)      # Muon eta
hmuon1_pt    = ROOT.TH1F("hmuon1_pt", "Muon pT", 80, 0, 100)           # Pt of the first muon
hmuon2_pt    = ROOT.TH1F("hmuon2_pt", "Muon pT", 80, 0, 100)           # Pt of the second muon
hdimuon_mass = ROOT.TH1F("hdimuon_mass", "Dimuon mass", 1000, 0, 150)  # Dimuon mass


# Define 2-dimensional simple histograms: TH1F(key, title, nbins, xmin, xmax)

hmuon_eta_phi = ROOT.TH2F("hmuon_eta_phi", "Muon phi vs eta", 80, -3.14, 3.14, 80, -2.4, 2.4)   # Leading vs trailing muon pt
hmuon_pt1_pt2 = ROOT.TH2F("hmuon_pt1_pt2", "Trailing vs leading muon pt", 80, 0, 100, 80, 0, 100)   # Leading vs trailing muon pt

## 2 - Histogram filling

We will use the histograms to study the collision data, so we will fill them with the data that we have in the file. In the example below, we loop over the events, we access the values of the physics observables that we want to study, and with these values we fill the histograms.

To loop over the events in the TTree ```ttree``` we run an iterable ```e``` from 0 to ```nEvents```. Every time that the call ```ttree.GetEntry(e)``` we access the TTree information for the event iterable number ```e```. The fill the histograms accordingly to what we want to represent in them:

- The muon histograms ```hmuon_pt```, ```hmuon_eta``` and ```hmuon_eta_phi``` are filled **per reconstructed muon**. That is, for every muon of every event.
- The dimuon histograms ```hmuon1_pt```, ```hmuon2_pt```, ```hdimuon_mass```, and ```hmuon_pt1_pt2``` are filled **once per event**, for every event that has at least two reconstructed muons.

Some notes:
- Some of the histograms are filled only if the events have at least two muons i.e. if they fulfill a given condition. This conditions are called *selection* or *cuts* that are applied to the events. They are used to select the physics phenomena that we want to explore.
- The number ```e``` is just an iterable with no meaning. The events can be idenfitied by the event id, which is a number stored as any other variable, that has nothing to do with that.
- Additionally, a progress bar is added to track the filling process. This is only for illustration, but not necessary, and not recommmended if running over a huge amount of data as it will make the loop slower.

In [ ]:
# Auxiliar progress bar to track the progress

import tqdm

pbar = tqdm.tqdm(total=nEvents) # Auxiliar progress bar

# Loop over TTree by accessing a given entry 
for e in range(0, nEvents):
    
    pbar.update(1)
    ttree.GetEntry(e)
    
    nMuon = ttree.nMuon
    hnMuon.Fill(nMuon)
    
    # Loop over all muons in the event
    for i in range(0, nMuon):
        
        hmuon_pt.Fill(ttree.Muon_pt[i])
        hmuon_eta.Fill(ttree.Muon_eta[i])
        hmuon_eta_phi.Fill(ttree.Muon_eta[i], ttree.Muon_phi[i])
    
    # Additionally, for events with at least two muons, we fill:
    if nMuon >= 2:
        
        hmuon1_pt.Fill(ttree.Muon_pt[0])
        hmuon2_pt.Fill(ttree.Muon_pt[1])
        
        mass = math.sqrt(2*ttree.Muon_pt[0]*ttree.Muon_pt[1]*(math.cosh(ttree.Muon_eta[0] - ttree.Muon_eta[1]) - math.cos(ttree.Muon_phi[0] - ttree.Muon_phi[1])))
        hdimuon_mass.Fill(mass)
        
        hmuon_pt1_pt2.Fill(ttree.Muon_pt[0], ttree.Muon_pt[1])

### Projecting a TTree into a histogram

In some occasions, if we don't need to apply much complex selections on our histograms, we can also fill them by using the method ```Project()```. This method is called on the TTree and it's telling it to project the data of a given branch (or alias) into an existing histogram. The histogram should be defined first with a distinct name. 

The advantage of this method is that it is much faster than looping over the events of the TTree.

This example shows how to project a couple of histograms:

In [ ]:
## Clone the histograms to project the TTree in the clones
hmuon_pt_projected = hmuon_pt.Clone("hmuon_pt_projected")
hdimuon_mass_projected = hmuon_pt.Clone("hdimuon_mass_projected")

## Reset the clones to remove existing entries before filling them again
hmuon_pt_projected.Reset()
hdimuon_mass_projected.Reset()

## Project muon pt
ttree.Project('hmuon_pt_projected', 'Muon_pt', '')

## Project dimuon mass by creating an alias first
ttree.SetAlias('Dimuon_mass', 'sqrt(2*Muon_pt[0]*Muon_pt[1]*(cosh(Muon_eta[0] - Muon_eta[1]) - cos(Muon_phi[0] - Muon_phi[1])))')
ttree.Project('hdimuon_mass_projected', 'Dimuon_mass', 'nMuon > 1')

In [ ]:
print("Entries of histograms from loop are: ", hmuon_pt.GetEntries(), hdimuon_mass.GetEntries())

print("Entries of histograms from projections are: ", hmuon_pt_projected.GetEntries(), hdimuon_mass_projected.GetEntries())

## 3 -  Histogram visualization

To visualize the histograms we need to plot them. Every plot in this example will be stored in a new created dir ```plots```. Additionally, we import some display tools to visualize them here.

In [ ]:
import os
from IPython.display import Image

if not os.path.exists('plots/'):
    os.mkdir('plots/')

### The canvas

The canvas is used to plot the histograms. Once created, the can call ```c1.cd()``` to indicate that this is the canvas in which we will plot them. If not created, a default canvas will be created automatically. However, if we want to manipulate the plots and later save them into images, it is necessary to define the canvas into a variable that we can controll:

In [ ]:
## Init the canvas

c1 = ROOT.TCanvas('c1', '', 600, 500)
c1.cd()

### Plotting a simple histogram

In this block of code we will just plot some histograms with the default ROOT style. For now, we just modify the axis title to indicate what is plotted.

To plot a histogram, we just call ```Draw()``` on the histogram we want to plot. Additionally, we can indicate how we want to show the histogram points. In the examples below we first plot the number of muons in points (option *"PE"*) and the muon transverse momentum pt in a line histogram (option *"HIST"*).

Both plots are saved in .png format by calling ```c1.SaveAs()``` indicating the name of the output png file.

Note that every time that we call ```Draw()``` on a new histogram the previous one is removed from the canvas.

In [ ]:
hnMuon.GetXaxis().SetTitle("Number of muons")
hnMuon.GetYaxis().SetTitle("Number of events")
hnMuon.Draw('PE')
c1.SaveAs('plots/hnMuon.png')
Image(filename='plots/hnMuon.png')

In [ ]:
hmuon_pt.GetXaxis().SetTitle("Muon p_{T} (GeV)") # Note the syntax
hmuon_pt.GetYaxis().SetTitle("Number of muons")
hmuon_pt.Draw('HIST')
c1.SaveAs('plots/hmuon_pt.png')
Image(filename='plots/hmuon_pt.png')

### Changing the style

When showing histograms, it is important that the information is well represented. In many cases, it is useful to change the marker size, line width and colors as in the example below, where we show exactly the same histograms but with a different style:

In [ ]:
hnMuon.SetMarkerStyle(20)
hnMuon.SetMarkerColor(ROOT.kBlack)
hnMuon.SetLineColor(ROOT.kBlack)
hnMuon.Draw('PE')
c1.SaveAs('plots/hnMuon_newStyle.png')
Image(filename='plots/hnMuon_newStyle.png')

In [ ]:
hmuon_pt.SetLineColor(ROOT.kRed+2)
hmuon_pt.SetLineWidth(2)
hmuon_pt.SetFillColor(ROOT.kRed-9)
hmuon_pt.Draw('HIST')
c1.SaveAs('plots/hmuon_pt_newStyle.png')
Image(filename='plots/hmuon_pt_newStyle.png')

### Plot two histograms together

It is very common to plot two histogramas together to show how different distributions compare with each other. For example, we may want to compare the same variable for two different subsets of data, or for data and simulation, of two different variables for the same subset of data.

This can be done by drawing two or more histograms sequentially by including the option *"SAME"*. This option will indicate the canvas to not remove the previous histogram(s) that were plotted there.

In the following block of code, we want to compare the pt of the first muon (labelled with index 1) and the pt of the second muon (labelled with index 0) stored in the events. We apply a different style to each histogram so we can distinguish them:

In [ ]:
## Plot the lepton number, comparing muons and electrons

hmuon1_pt.GetXaxis().SetTitle("Muon p_{T} (GeV)")
hmuon1_pt.GetYaxis().SetTitle("Number of events")
hmuon1_pt.SetLineColor(ROOT.kRed)
hmuon1_pt.SetLineWidth(2)
hmuon2_pt.SetLineColor(ROOT.kBlue)
hmuon2_pt.SetLineWidth(2)

hmuon1_pt.Draw('HIST')
hmuon2_pt.Draw('HIST, SAME') # NOTICE: The word same allows for plotting in the canvas without clearing it first

c1.SaveAs('plots/hmuon_pt_comp.png')
Image(filename='plots/hmuon_pt_comp.png')

In the image above, we clearly see that one histogram is lying out of the canvas frame. This is becase the frame is adapted to the first histogram that is drawn. We can correct it by identifying the maximum value of both histograms and setting the value of the first one to that maximum value (with a little 20% marging). Then we can draw them again:

In [ ]:
# Check the maximum values of the histograms
print("The maximum value for the first muon is %f"%hmuon1_pt.GetMaximum())
print("The maximum value for the second muon is %f"%hmuon2_pt.GetMaximum())

maxValue = max([hmuon1_pt.GetMaximum(), hmuon2_pt.GetMaximum()])

hmuon1_pt.SetMaximum(1.2*maxValue)
hmuon1_pt.Draw('HIST')
hmuon2_pt.Draw('HIST, SAME')

c1.SaveAs('plots/hmuon_pt_comp.png')
Image(filename='plots/hmuon_pt_comp.png')

### The legend

Still, if an outsider sees this histogram, they won't be able to identify which are the distributions that are shown. For this, we can add a legend as the example below:

In [ ]:
# Legend object:
legend = ROOT.TLegend(0.15,0.7,0.48,0.85)
legend.SetBorderSize(0) # What is this for? Check it!
legend.SetTextSize(0.035) # Try to change it
legend.AddEntry(hmuon1_pt,"Leading muon","l")
legend.AddEntry(hmuon2_pt,"Trailing muon","l")

# Draw it, update the canvas and save it again:
legend.Draw()
c1.Update()
c1.SaveAs('plots/hmuon_pt_comp.png')
Image(filename='plots/hmuon_pt_comp.png')

### Improve the visualization

Now we can look at other variables, as the dimuon invariant mass. We plot it following the same steps as before: 

In [ ]:
hdimuon_mass.GetXaxis().SetTitle("Dimuon invariant masss m_{#mu#mu} (GeV)") # Note the syntax
hdimuon_mass.GetYaxis().SetTitle("Number of events")
hdimuon_mass.SetLineColor(ROOT.kBlack)
hdimuon_mass.Draw('HIST')
c1.SaveAs('plots/hdimuon_mass.png')
Image(filename='plots/hdimuon_mass.png')

We can see how this variable is distributed following a very specific shape. There is some continuum but also some peaks. These peaks are representative of particle dimuon resonances i.e. particles that decay into the two muons that we are using to reconstruct the invariant mass.

As the histogram covers many events in the low mass regime and limited statistics, it may be useful to explore this histogram in logarithm scale:

In [ ]:
c1.SetLogy(1)
c1.SaveAs('plots/hdimuon_mass_logY.png')
Image(filename='plots/hdimuon_mass_logY.png')

Now the peak at 90 GeV clearly, but the ones at low mass are not resolved. To better explore them, we can also try to set a logarithmic scale in the x axis.

To note: This histogram is defined from 0 to 140, so the 0 is not going to be well-defined in a logarithmic x axis. ROOT will asign a minimum value automatically, but we can also do it by defining the range of the histogram with ```.GetXaxis().SetRangeUser(0.4, 140.0)```:

In [ ]:
c1.SetLogx(1)
hdimuon_mass.GetXaxis().SetRangeUser(0.4, 140.0)
c1.SaveAs('plots/hdimuon_mass_logY_logX.png')
Image(filename='plots/hdimuon_mass_logY_logX.png')

In all the examples above we can see a box in the upper right corner with some information of the histogram i.e. the number of entries, the mean and std deviation of the data included in the histogram. Since this box is many times irrelevant, we will want to remove it and we can do it by running:

In [ ]:
ROOT.gStyle.SetOptStat(0)
c1.SaveAs('plots/hdimuon_mass_logY_logX_noStat.png')
Image(filename='plots/hdimuon_mass_logY_logX_noStat.png')

### 2-Dimensional plots



In [ ]:
# Set the axis to linear scale again
c1.SetLogx(0)
c1.SetLogy(0)

# Define the axis names for the 2-dimensional histogram
hmuon_eta_phi.GetXaxis().SetTitle("Muon #eta")
hmuon_eta_phi.GetYaxis().SetTitle("Muon #phi")
hmuon_eta_phi.Draw('COLZ')

# Save an show
c1.SaveAs('plots/hmuon_eta_phi.png')
Image(filename='plots/hmuon_eta_phi.png')

### Check the histograms and save them

In [ ]:
from IPython.display import Image

toDisplay = 'plots/hmuon_pt_comp.png'

Image(filename=toDisplay)

## 4 - Saving the results

Since plotting a histograms requires the corresponding TH1F object (and it usually takes time to fill), it is always recomended to save it into a ROOT file that can be opened later. 

To do so, we will have to create the ROOT file, with the option ```"RECREATE"```. Then, we will just have to point to this file with the ```cd()``` method and ```Write()``` each histogram. Finally, the file can be closed.

In [ ]:
output = ROOT.TFile("output.root", "RECREATE")
output.cd()

hnMuon.Write()
hmuon_pt.Write()
hmuon_eta.Write()
hmuon1_pt.Write()
hmuon2_pt.Write()
hdimuon_mass.Write()
hmuon_eta_phi.Write()
hmuon_pt1_pt2.Write()

output.Close()